In [1]:
import os
import subprocess
import multiprocessing 
import threading
from functools import partial
from PIL import Image

In [2]:
def makedir(dirName):
    cmd="mkdir {}"
    os.system(cmd.format( dirName))

In [3]:
thCount=multiprocessing.cpu_count()
thCount

12

In [4]:
#To further improve efficiency of the code, make the process detachable
#to make this function detachable, remove all the use of global variable, i.e. new/oldDirName, folder
#possibly by making them input parameters
def linkImg(ID, directoryList):
    
    print(directoryList[ID]+' ')
    #create a new subfolder in new directory
    makedir(os.path.join(longPath, newDirName, folder, '"'+directoryList[ID]+'"'))
    #in the subfolder of old directory
    fullPath=os.path.join(longPath, oldDirName, folder, directoryList[ID])
    elementList=os.listdir(fullPath)
    if('.DS_Store' in elementList):
        elementList.remove('.DS_Store')
    
    #the following parameter is prepared seperate clip
    #timeMark for labeling frame No
    timeMark=''
    elementList.sort()
    
    nameOfClip='clip{}'
    seperateClip=True
    #for data augmentation
    selectResidue=[0]
    loopCount=1
    lncmd="ln -sf {} {}"
    
    redoDir=False
    #insert code for data augmentation for training only
    
    if(directoryList[ID] in tinyList and folder == 'Training'):
        selectResidue=[0,1,2,3,4]
        loopCount=5
    elif(directoryList[ID] in smallList and folder == 'Training'):
        selectResidue=[0,2,4]
        loopCount=3
    elif(directoryList[ID] in mediumList and folder == 'Training'):
        selectResidue=[0,2]
        loopCount=2
    for it in range(loopCount):
        #iterator for clip naming
        iterator=0
        timeMark=''
        if it==0:
            extraLoop=False
        else:
            extraLoop=True
        prevSubkNo=''
        for element in elementList:
            if(int(element[4:10])%frameGap!=selectResidue[it]):
                continue
            #check subject number, if doesn't exist, create new subject folder under new directory
            subjectNo=element[:2]
            if(not os.path.exists(os.path.join(longPath, newDirName, folder, directoryList[ID], subjectNo))):
                #print('creating folder')
                timeMark='';
                iterator=0
                makedir(os.path.join(longPath, newDirName, folder, '"'+directoryList[ID]+'"', subjectNo))
            elif extraLoop==True and prevSubkNo!=subjectNo:
                tmpLst=os.listdir(os.path.join(longPath, newDirName, folder, directoryList[ID], subjectNo))
                tmpLst.sort()
                iterator=int(tmpLst[-1][4:])+1
                prevSubkNo=subjectNo
                redoDir=True
#                 print('iterator: ',iterator)
#                 print('curList: ',tmpLst)
#                 print('subjNo', subjectNo)
#                 print('******')
        
        
        
            #determine if need to seperate clips from clips
            if(seperateClip):
                #if is start of series
                if timeMark == '':
                    NOP=nameOfClip.format(iterator)
                    makedir(os.path.join(longPath, newDirName, folder, '"'+directoryList[ID]+'"', subjectNo, NOP))
                    redoDir=False
                elif redoDir==True:
                    NOP=nameOfClip.format(iterator)
                    makedir(os.path.join(longPath, newDirName, folder, '"'+directoryList[ID]+'"', subjectNo, NOP))
                    redoDir=False
                else:
                    #if gap exist
                    if(abs(int(elemetinyList=['CHEST-TUBE PREP','KING AIRWAY','SPLINTING']
smallList=['BLOOD-PRESSURE CUFF','COMBAT GAUZE','COMBAT TOURNIQUET','ECG LEADS','INTUBATION','IO LINE','PULSE-OX','VITAL CHECKING','WRAP HEAD WOUND']
mediumList=['BAGGING','CHEST-TUBE','IM ADMINISTRATION','IV TOURNIQUET','SUTURING','IV LINE']nt[6:10])-int(timeMark))>2*frameGap):
                        iterator+=1
                        NOP=nameOfClip.format(iterator)
                        makedir(os.path.join(longPath, newDirName, folder, '"'+directoryList[ID]+'"', subjectNo, NOP))
                    #else
                #link original to destination
                os.system(lncmd.format( os.path.join(longPath, oldDirName, folder, '"'+directoryList[ID]+'"', element), 
                        os.path.join(longPath, newDirName, folder, '"'+directoryList[ID]+'"', subjectNo, NOP, element)))
                timeMark=element[6:10]
            else:        
            #make softlink for the element
        
                os.system(lncmd.format( os.path.join(longPath, oldDirName, folder, '"'+directoryList[ID]+'"', element), 
                        os.path.join(longPath, newDirName, folder, '"'+directoryList[ID]+'"', subjectNo, element)))

In [6]:
newDirName='NewResizedBinned'
oldDirName='NewResized'
longPath='/media/livelab/DataStorage/Fold5'
cmd="mkdir {}"
ignoreList=['CLAP','INCISION','INTUBATION-STYLETE','MEDICATION','PLACE CUFF']
tinyList=['CHEST-TUBE PREP','PULSE-OX','KING AIRWAY','ORAL AIRWAY']
smallList=['BLOOD-PRESSURE CUFF','COMBAT GAUZE','COMBAT TOURNIQUET','ECG LEADS','INTUBATION','IO LINE','PULSE-OX','VITAL CHECKING','WRAP HEAD WOUND']
mediumList=['BAGGING','CHEST-TUBE','IM ADMINISTRATION','IV TOURNIQUET','SUTURING','IV LINE']
frameGap=5
makedir(os.path.join(longPath,newDirName))
#symbol link from the original file to new location after classification
#symbol link example: ln -sf source destination
folderList=['Training','Validation','Test']

for folder in folderList:
    print('Performing operation for '+ folder+'\n')
    if os.path.exists(os.path.join(longPath, newDirName, folder)):
        print('Folder '+ folder +' already exists, skip\n')
        continue
    makedir(os.path.join(longPath, newDirName, folder))
    target="{}/{}".format(oldDirName, folder)
    dirList=os.listdir(os.path.join(longPath, target))
    if('.DS_Store' in dirList):
        dirList.remove('.DS_Store')
    for cate in dirList:
        if cate in ignoreList:
            dirList.remove(cate)
    thCount=multiprocessing.cpu_count()
    pool = multiprocessing.Pool(thCount-4)
    partial_linkImg=partial(linkImg, directoryList=dirList)
    N = len(dirList)
    _=pool.map(partial_linkImg,range(N))
    pool.close()
    pool.join()

Performing operation for Training

CHEST-TUBE PREP 
INTUBATION 
SUTURING 
CPR (COMPRESSION) 
PULSE-OX 
COMBAT TOURNIQUET 
ECG LEADS 
DRAW MEDICATION 
SWAB AREA WITH ALCOHOL 
WRAP HEAD WOUND 
IV TOURNIQUET 
SPLINTING 
IV LINE 
IO LINE 
KING AIRWAY 
CHEST-TUBE 
VITAL CHECKING 
ADMINISTER MEDICATION 
IM ADMINISTRATION 
BAGGING 
CPR (BREATH) 
ORAL AIRWAY 
BLOOD-PRESSURE CUFF 
COMBAT GAUZE 
Performing operation for Validation

DRAW MEDICATION 
CHEST-TUBE PREP 
INTUBATION 
PULSE-OX 
SUTURING 
CPR (COMPRESSION) 
COMBAT TOURNIQUET 
ECG LEADS 
SWAB AREA WITH ALCOHOL 
WRAP HEAD WOUND 
IV TOURNIQUET 
SPLINTING 
IV LINE 
IO LINE 
KING AIRWAY 
CHEST-TUBE 
VITAL CHECKING 
ADMINISTER MEDICATION 
IM ADMINISTRATION 
BAGGING 
CPR (BREATH) 
ORAL AIRWAY 
BLOOD-PRESSURE CUFF 
COMBAT GAUZE 
Performing operation for Test

CPR (COMPRESSION) 
SUTURING 
CHEST-TUBE PREP 
INTUBATION 
ECG LEADS 
COMBAT TOURNIQUET 
PULSE-OX 
SWAB AREA WITH ALCOHOL 
DRAW MEDICATION 
WRAP HEAD WOUND 
IV TOURNIQUET 
SPLINTING 
IV LINE

In [ ]:
#old code

In [4]:
newDirName='ResizeBinned'
oldDirName='Resize'
cmd="mkdir {}"
makedir(newDirName)
#symbol link from the original file to new location after classification
#symbol link example: ln -sf source destination
folderList=['Test','Training' , 'Validation']
longPath='/Users/lingfengli/Desktop/SIR/cela-tf-starter'
for folder in folderList:
    print('Performing operation for '+ folder+'\n')
    makedir(os.path.join(longPath, newDirName, folder))
    target="{}/{}".format(oldDirName, folder)
    dirList=os.listdir(os.path.join(longPath, target))
    if('.DS_Store' in dirList):
        dirList.remove('.DS_Store')
    #a processed dirList escaping space character
    dirListP=[]
#     for folderName in dirList:
#         folderName=folderName.replace(" ", "\" \"")
#         dirListP.append(folderName)
#     for subfolder in dirListP:
        
    for subfolder in dirList:
        print(subfolder+' ')
        #create a new subfolder in new directory
        makedir(os.path.join(longPath, newDirName, folder, '"'+subfolder+'"'))
        #in the subfolder of old directory
        fullPath=os.path.join(longPath, oldDirName, folder, subfolder)
        elementList=os.listdir(fullPath)
        if('.DS_Store' in elementList):
            elementList.remove('.DS_Store')
        for element in elementList:
            #check subject number, if doesn't exist, create new subject folder under new directory
            subjectNo=element[:2]
            if(not os.path.exists(os.path.join(longPath, newDirName, folder, subfolder, subjectNo))):
                #print('creating folder')
                makedir(os.path.join(longPath, newDirName, folder, '"'+subfolder+'"', subjectNo))
            #make softlink for the element
            lncmd="ln -sf {} {}"
            os.system(lncmd.format( os.path.join(longPath, oldDirName, folder, '"'+subfolder+'"', element), 
                           os.path.join(longPath, newDirName, folder, '"'+subfolder+'"', subjectNo, element)))

Performing operation for Test

CPR (BREATH) 


In [83]:
folderList=['Test','Training' , 'Validation']
longPath='/Users/lingfengli/Desktop/SIR/cela-tf-starter'
# for folder in folderList:
#     target="/{}/{}".format(oldDirName, folder)
#     print(target)
os.system(cmd.format( os.path.join(longPath, newDirName, folderList[0])))
target="{}/{}".format(oldDirName, folderList[0])
print(target)


Resize/Test


In [115]:
dirList=os.listdir(os.path.join(longPath, target))
if('.DS_Store' in dirList):
    dirList.remove('.DS_Store')
#a processed dirList escaping space character
dirListP=[]
for folderName in dirList:
    folderName=folderName.replace(" ", "\" \"")
    dirListP.append(folderName)
for subfolder in dirListP:
    #create a new subfolder in new directory
    os.system(cmd.format( os.path.join(longPath, newDirName, folderList[0], subfolder)))
# for subfolder in dirList:    
#     #in the subfolder of old directory
#     fullPath=os.path.join(longPath, oldDirName, folderList[0], subfolder)
#     elementList=os.listdir(fullPath)



In [121]:
fullPath=os.path.join(longPath, oldDirName, folderList[0], dirList[0])
elementList=os.listdir(fullPath)
if('.DS_Store' in elementList):
    elementList.remove('.DS_Store')

In [134]:
for element in elementList:
    #check subject number, if doesn't exist, create new subject folder under new directory
    subjectNo=element[0:2]
    if(not os.path.exists(os.path.join(longPath, newDirName, folderList[0], dirList[0], subjectNo))):
        print('creating folder')
        makedir(os.path.join(longPath, newDirName, folderList[0], dirList[0], subjectNo))
    #make softlink for the element
    lncmd="ln -sf {} {}"
    os.system(lncmd.format( os.path.join(longPath, oldDirName, folderList[0], dirList[0], element), 
                           os.path.join(longPath, newDirName, folderList[0], dirList[0], subjectNo, element)))

In [1]:
a=['abc1','abc2','abc3']
a.sort()
a

['abc1', 'abc2', 'abc3']

In [10]:
int(a[4:])

123

In [5]:
123%10

3